In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit 
from teslakit.database import Database
from teslakit.io.matlab import ReadMatfile
from teslakit.rbf import RBF_Reconstruction


In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('GUAM')


In [4]:
# --------------------------------------
# load data and set parameters

# TCs copula simulated (dataset) and TCs MDA selection (NO SE USA!!!!!!)
xds_dataset = db.Load_TCs_r2_sim_params()
xds_subset = db.Load_TCs_r2_mda_params()

# r2 mda TCs solved simulations (not solved inside teslakit)
xds_TCs_sims = db.Load_TCs_r2_mda_Simulations() # 400 simulaciones de olas correspondientes a los 400 centroides de maxdis
print(xds_TCs_sims)


<xarray.Dataset>
Dimensions:  (storm: 400)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    hs       (storm) float64 1.182 12.17 0.01416 0.3082 ... 7.627 0.1468 1.15
    tp       (storm) float64 9.283 13.89 1.499 5.714 ... 11.76 12.15 11.16 10.69
    dir      (storm) float64 235.2 42.54 239.0 13.96 ... 124.5 182.8 355.5 346.2
    ss       (storm) float64 0.002797 0.6178 0.007105 ... 0.239 0.01714 0.0224
    twl      (storm) float64 0.5377 3.186 0.01656 0.1713 ... 2.018 0.2438 0.6297
    mu       (storm) float64 0.5616 0.6201 0.7014 ... 0.8513 0.2757 0.3203


In [5]:
# --------------------------------------
# load custom RBFs training subset and dataset to interpolate
# TODO: our datasets do not match with current solved simulations 

# we are using matching matlab files for KWAJALEIN TCs simulations
p_test = op.join(p_data, 'tests', 'RBF_GUAM')
p_subset = op.join(p_test, 'MDA_400_GUAM.mat')
p_dataset = op.join(p_test, 'MULTIVARIATE_100000parameters.mat')

# load subset data
subset = ReadMatfile(p_subset)['Subset']  # pmean, vmean, gamma, delta

# load full dataset for interpolation
dm = ReadMatfile(p_dataset)
dataset= np.column_stack(
    (dm['PMEAN1'], dm['VMEAN'], dm['GAMMA'], dm['DELTA'])
)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/albacid/Projects/TeslaKit_projects/tests/RBF_GUAM/MULTIVARIATE_100000parameters.mat'

In [ ]:
# --------------------------------------
# Extract data from TCs simulations (this is the RBFs training target)

print(xds_TCs_sims)
print()

# Normalize data
d_maxis = {}
d_minis = {}
xds_n = xds_TCs_sims.copy()
for k in ['hs', 'tp', 'ss', 'twl']:
    v = xds_n[k].values[:]
    mx = np.max(v)
    mn = np.min(v)
    xds_n[k] =(('storm',), (v-mn)/(mx-mn))
    
    # store maxs and mins for denormalization
    d_maxis[k] = mx
    d_minis[k] = mn

xds_n['dir'] = xds_n['dir'] * np.pi/180
    
print(xds_n)
print()


# Build RBF target numpy array
target = np.column_stack(
    (xds_n['hs'], xds_n['tp'], xds_n['ss'], xds_n['twl'], xds_n['dir'], xds_n['mu'])
)


In [ ]:
# --------------------------------------
# RBF Interpolation

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]        # scalar (pmean,  vmean)
ix_directional_subset = [2,3]   # directional (delta, gamma)

# target - scalar / directional indexes
ix_scalar_target = [0,1,2,3,5]  # scalar (Hs, Tp, SS, TWL, MU)
ix_directional_target = [4]     # directional (Dir)

output = RBF_Reconstruction(
    subset, ix_scalar_subset, ix_directional_subset,
    target, ix_scalar_target, ix_directional_target,
    dataset)


In [ ]:
# --------------------------------------
# output de-normalization

hs_n = output[:,0]
tp_n = output[:,1]
ss_n = output[:,2]
twl_n = output[:,3]
dir_n = output[:,4]
mu = output[:,5]

# store data as xarray.Dataset
xds_out = xr.Dataset(
    {
        'hs':(('storm',), hs_n * (d_maxis['hs']-d_minis['hs']) + d_minis['hs'] ),
        'tp':(('storm',), tp_n * (d_maxis['tp']-d_minis['tp']) + d_minis['tp'] ),
        'ss':(('storm',), ss_n * (d_maxis['ss']-d_minis['ss']) + d_minis['ss'] ),
        'twl':(('storm',), twl_n * (d_maxis['twl']-d_minis['twl']) + d_minis['twl'] ),
        'dir':(('storm',), dir_n * 180 / np.pi),
        'mu':(('storm',), mu),
    },
    coords = {'storm': np.arange(output.shape[0])}
)
print(xds_out)

db.Save_TCs_sim_r2_rbf_output(xds_out)
